In [28]:
import celltk
import numpy as np
from collections import Counter

In [2]:
arr = celltk.ExperimentArray.load('example.hdf5')

## Density of cells at each location in initial frame

In [15]:
fig = celltk.PlotHelper().heatmap2d_plot(np.concatenate(arr['nuc', 'fitc', 'x', :, 0]).ravel(),
                                         np.concatenate(arr['nuc', 'fitc', 'y', :, 0]).ravel(),
                                         xbinsize=20, ybinsize=20, colorscale='Greys')
fig.show()

## Distribution of nuclear areas for whole experiment

In [6]:
fig = celltk.PlotHelper().histogram_plot(arr['nuc', 'tritc', 'area', :, 0], 
                                         include_kde=True, include_histogram=False,
                                         keys=arr.conditions, histnorm='probability density')
fig.show()

## Mean nuclear marker intensity over time

In [8]:
fig = celltk.PlotHelper().line_plot(arr['nuc', 'tritc', 'median_intensity'], 
                                    estimator=np.nanmean, time=arr.time,
                                    keys=arr.conditions)
fig.show()

/Users/sjeknic/.pyenv/versions/3.9.9/envs/celltk_pip/lib/python3.9/site-packages/numpy/lib/shape_base.py:402: RuntimeWarning:

Mean of empty slice



## Distributions of mean distance travelled per frame

In [33]:
conds = [c for c in arr.conditions if 'il6' not in c and 'torin' not in c]
diffx = [np.diff(x, axis=1) ** 2 for x in arr[conds]['nuc', 'fitc', 'x']]
diffy = [np.diff(y, axis=1) ** 2 for y in arr[conds]['nuc', 'fitc', 'y']]

diff = [(x + y) ** 0.5 for x, y in zip(diffx, diffy)]
fig = celltk.PlotHelper().histogram_plot([np.nanmean(d, axis=1) for d in diff], 
                                         include_histogram=False, include_kde=True, 
                                         keys=conds, histnorm='probability density',
                                         colors=['dimgray'], alpha=0.5)


conds = [c for c in arr.conditions if 'il6' in c and 'torin' not in c]
diffx = [np.diff(x, axis=1) ** 2 for x in arr[conds]['nuc', 'fitc', 'x']]
diffy = [np.diff(y, axis=1) ** 2 for y in arr[conds]['nuc', 'fitc', 'y']]

diff = [(x + y) ** 0.5 for x, y in zip(diffx, diffy)]
fig = celltk.PlotHelper().histogram_plot([np.nanmean(d, axis=1) for d in diff], 
                                         include_histogram=False, include_kde=True, 
                                         keys=conds, histnorm='probability density',
                                         colors=['magenta'], figure=fig)
fig.show()


## Mean nuclear marker intensity in initial frame by location in field of view

In [32]:
# Collect data and remove nans
x = np.concatenate(arr['nuc', 'fitc', 'x', :, 0]).ravel()
y = np.concatenate(arr['nuc', 'fitc', 'y', :, 0]).ravel()
i = np.concatenate(arr['nuc', 'tritc', 'median_intensity', :, 0]).ravel()
nan_mask = np.logical_or(np.isnan(x), np.isnan(y))
x = x[~nan_mask]
y = y[~nan_mask]
i = i[~nan_mask]

# Discretize data
xbins = np.histogram_bin_edges(np.arange(853), bins=39)
ybins = np.histogram_bin_edges(np.arange(720), bins=39)
xdig = np.digitize(x, bins=xbins, right=False)
ydig = np.digitize(y, bins=ybins, right=False)
zdim = Counter(zip(xdig,ydig)).most_common()[0][1]

# Make destination array
temp = np.empty((40, 40, zdim))
temp[:] = np.nan

# Transfer data
for x, y, val in zip(xdig, ydig, i):
    z = np.argmax(np.isnan(temp[x, y, :]))
    temp[x, y, z] = val

fig = celltk.PlotHelper().heatmap_plot(np.nanmean(temp, axis=-1), 
                                       zmin=1000, zmax=3000, colorscale='Greys')
fig.show()



/var/folders/2j/x9t3x68j27v37txf6kq8096h0000gn/T/ipykernel_74966/1917657071.py:26: RuntimeWarning:

Mean of empty slice

